### Exercise 1-3. 

This exercise deals with some issues of ‘‘sensitivity’’ in the steel models.

(a) For the linear program of Figures 1-5a and 1-5b, display Time and Make.rc. What do these values tell you about the solution? (You may wish to review the explanation of marginal values and reduced costs in Section 1.6.)

(b) Explain why the reheat time constraints added in Figure 1-6a result in a higher production of plate and a lower production of bands.

(c) Use AMPL to verify the following statements: If the available reheat time is increased from 35 to 36 in the data of Figure 1-6b, then the profit goes up by $\$$1800 as predicted in Section 1.6. If the reheat time is further increased to 37, the profit goes up by another $\$$1800. However, if the reheat time is increased to 38, there is a smaller increase in the profit, and further increases past 38 have no effect on the optimal profit at all. To change the reheat time to, say, 26 without changing and reading the data file over again, type the command let avail["reheat"] := 36; By trying some other values of the reheat time, confirm that the profit increases by $\$$1800 per extra hour for any number of hours between 35 and 37 9/14, but that any increase in the reheat time beyond 37 9/14 hours doesn’t give any further profit. Draw a plot of the profit versus the number of reheat hours available, for hours ≥ 35.

(d) To find the slope of the plot from (c) — profit versus reheat time available — at any particular reheat time value, you need only look at the marginal value of Time["reheat"]. Using this observation as an aid, extend your plot from (c) down to 25 hours of reheat time. Verify that the slope of the plot remains at $6000 per hour from 25 hours down to less than 12 hours of reheat time. Explain what happens when the available reheat time drops to 11 hours

In [102]:
import pandas as pd
from amplpy import AMPL, Environment

In [103]:
ampl = AMPL(Environment('/opt/ampl.linux64'))

In [136]:
ampl.reset()

In [137]:
ampl.read('../Model3c.mod')

In [138]:
ampl.readData('../Model3c.dat')

In [139]:
variables = ('bands','coils','plate')
time_constraints = ()

In [140]:
print(ampl.getConstraint('Time'))
#print(ampl.getConstraint('Time').get('reheat'))
#print(ampl.getConstraint('Time').get('roll'))
print()
print('Variables and their bounds')
for i in ampl.getVariables():
    [print(i[1].get(j)) for j in variables]
print()
print('Constraints and their limits')
for i in ampl.getConstraints():
    if time_constraints == (): 
        print(i[1].get())
    #if i[0] == 'Time': [print(i[1].get(j)) for j in time_constraints]
    #if i[0] == 'max_weight': print(i[1].get())
    #if i[0] == 'min_tons': [print(i[1].get(j)) for j in variables]
print()
print('Objective Function')
print(ampl.getObjective('Total_Profit').get())

subject to Time: sum{p in Products} 1/tonsPerHour[p]*X[p] <= maxHours;

Variables and their bounds
var X['bands'] >=1000, <=6000;
var X['coils'] >=500, <=4000;
var X['plate'] >=750, <=3500;

Constraints and their limits
subject to Time:
	0.005*X['bands'] + 0.00714286*X['coils'] + 0.00625*X['plate'] <= 40;

Objective Function
maximize Total_Profit:
	25*X['bands'] + 30*X['coils'] + 29*X['plate'];


In [141]:
ampl.eval('expand Total_Profit;')
ampl.eval('expand Time;')

maximize Total_Profit:
	25*X['bands'] + 30*X['coils'] + 29*X['plate'];

subject to Time:
	0.005*X['bands'] + 0.00714286*X['coils'] + 0.00625*X['plate'] <= 40;



In [142]:
for variable in variables:
    ampl.eval('expand X["{}"];'.format(variable))

Coefficients of X['bands']:
	Time           0.005
	Total_Profit  25

Coefficients of X['coils']:
	Time           0.00714286
	Total_Profit  30

Coefficients of X['plate']:
	Time           0.00625
	Total_Profit  29



In [143]:
ampl.eval('display _nvars, _ncons;')

_nvars = 3
_ncons = 1



In [144]:
print(ampl.getOption('solver'))
ampl.setOption('solver','cplex')
print(ampl.getOption('solver'))

cplex
cplex


In [145]:
ampl.solve()

CPLEX 12.8.0.0: optimal solution; objective 194828.5714
1 dual simplex iterations (0 in phase I)


In [146]:
print(ampl.getObjective('Total_Profit').get().value())

194828.57142857142


#### Shadow Price or Dual Price or Marginal Price

In [147]:
ampl.eval('display Total_Profit;')
for i in ampl.getConstraints():
    print(i[1].getValues())
#print(ampl.getConstraint('Time').getValues())

Total_Profit = 194829

 Time.dual  
    4640    



### Decision Variable Solution and Reduced Cost

In [148]:
X_result = list()
for p in variables:
    X_result.append([
        p,
        ampl.getVariable('X').get(p).lb(), 
        ampl.getVariable('X').get(p).value(),
        ampl.getVariable('X').get(p).ub(),
        ampl.getVariable('X').get(p).rc()
    ])
df_X_result = pd.DataFrame(X_result)
df_X_result.rename(columns={0:'DecisionVariable_X',
                            1:'X_LowerBound',
                            2:'X_Solution',
                            3:'X_UpperBound',
                            4:'X_ReducedCosts'},
                   inplace=True)
df_X_result.set_index('DecisionVariable_X',inplace=True)
df_X_result

,X_LowerBound,X_Solution,X_UpperBound,X_ReducedCosts
DecisionVariable_X,,,,
bands,1000.0,6000.000000,6000.0,1.800000
coils,500.0,500.000000,4000.0,-3.142857
plate,750.0,1028.571429,3500.0,0.000000


In [149]:
print(ampl.getData('X'))
values = ampl.getVariable('X').getValues()
print(values)
df = values.toPandas()
print(values.toList())
print(values.toDict())

   index0    |      X      
  'bands'    |     6000    
  'coils'    |     500     
  'plate'    | 1028.5714285714287

   index0    |    X.val    
  'bands'    |     6000    
  'coils'    |     500     
  'plate'    | 1028.5714285714287

[('bands', 6000.0), ('coils', 500.0), ('plate', 1028.5714285714287)]
{'bands': 6000.0, 'coils': 500.0, 'plate': 1028.5714285714287}


In [150]:
df['X.rollRate'] = [1/i for i in (200,140,160)]
df['X.roll']=df['X.val']*df['X.rollRate']
df['X.ratio']=df['X.val']/df['X.val'].sum()
df

,X.val,X.rollRate,X.roll,X.ratio
bands,6000.000000,0.005000,30.000000,0.796964
coils,500.000000,0.007143,3.571429,0.066414
plate,1028.571429,0.006250,6.428571,0.136622


In [151]:
df_ = ampl.getEntity('dollarProfitPerTon').getValues().toPandas()
df_ = df.join(df_)
print('Roll:{}'.format(df['X.roll'].sum()))
print('Total Production = {} tons'.format(df['X.val'].sum()))
print('total profit under new objective function = {}'.format((df_.loc[:,'X.val'] * df_.loc[:,'dollarProfitPerTon']).sum()))

Roll:40.0
Total Production = 7528.571428571428 tons
total profit under new objective function = 194828.57142857142


## Summary

(a) Interpretation of Shadow Price/Dual Price and Reduced Cost:

1. Shadow Price (Dual Price): The objective function has value of $\$$194828.57 and the Time constraint, has a shadow price of $\$$4640. This means that if we increased the time constraint limit of 40 hours by 1 hour to 41 hour, then the objective function (total profit) would increase from $\$$194828.57 to $\$$199468.57 ($\$$199468.57-$\$$194828.57 = $\$$4640.0)

2. Reduced Cost: Firstly reduced cost applies to decision variables. Reduced cost is non zero for decision variables that are solved at their min or max bounds. In this case bands are at max of 6000 tons and has a reduced cost of $\$$1.8 while coils are at min of 500 tons and has a reduced cost of $\$$-3.14. 

    * If max bound of bands was increased by 1 unit from 6000 tons to 6001, this would increase the objective function (total profit) from $\$$194828.57 to $\$$194830.37 ($\$$194830.37-$\$$194828.57=$\$$1.8=Reduced Cost of 699).
    * If min bound of coils was increased by 1 unit from 500 tons to 501, this would decrease the objective function (total profit) from $\$$194828.57142857142 to $\$$194825.42857142858 ($\$$194828.57142857142-$\$$194825.42857142858=$\$$3.142857142840512).

In [117]:
194825.42857142858-194828.57142857142

-3.142857142840512